### Reference

**Initialize Parameters** :  
https://towardsdatascience.com/coding-a-2-layer-neural-network-from-scratch-in-python-4dd022d19fd2



In [8]:
x = 100
y = x
y = 50
x = y
print(x,y)

50 50


In [ ]:
class TwoLayerNet:
    def __init__(self, input_size:int, hidden_size:int, output_size:int, hidden_layer_act:str, output_layer_act:str, weight_init_std=.01):
        # Initialize parameters
        self.params = {}
        self.params['w1'] = np.random.randn(input_size, hidden_size)/np.sqrt(input_size)
        self.params['b1'] = np.zeros((hidden_size, 1))
        self.params['w2'] = np.random.randn(hidden_size, output_size)/np.sqrt(hidden_size)
        self.params['b2'] = np.random.randn((output_size, 1))
        
        # Build layers
        self.layers = OrderedDict()
        
        self.layers['Linear1'] = Linear(self.params['w1'],self.params['b1'])
        if hidden_layer_act == 'ReLU':
            self.layers['ReLU_1'] = ReLU()
        elif hidden_layer_act == 'Sigmoid':
            self.layers['Sigmoid1'] = Sigmoid()
        elif hidden_layer_act == 'tanh':
            self.layers['tanh1'] = tanh()
        elif hidden_layer_act == 'Leaky ReLU':
            self.layers['LeakyReLu1'] = Leaky_ReLU()
            
        self.layers['Linear2'] = Linear(self.params['w2'],self.params['b2'])
        self.output_layer = SoftmaxwithCrossEntropyLoss()
        
    def predict(self, x):  # x starts with data features and update through layers
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, y):
        yhat = self.predict(x)
        return self.output_layer.forward(yhat, y)
    
    

In [ ]:
class Linear:  #book P137+ dimension reshape
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dw = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # (n_samples, 1d)
        self.x = x
        summ = np.dot(self.x, self.w) + self.b
        return summ
    
    def backward(self, dout):   # need math handwrt
        dx = np.dot(dprev, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dprev, axis=0)  # ?
        dx = dx.reshape(*self.original_x_shape)  # ???
        return dx
    
class ReLU: #book P128
    def __init__(self, w, b):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
class Sigmoid: #book P129
    def __init__(self, w, b):
        self.out = None
        
    def forward(self, x):
        out = 1/(1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
    
class 

In [5]:
# import numpy as np
x = np.array([[1,2,3],[3,4,5]])
y = x.shape # (2,3)
x = x.reshape(1,6)
x.reshape(*y)

array([[1, 2, 3],
       [3, 4, 5]])

In [6]:
# The special syntax *args in function definitions in python 
# is used to pass a variable number of arguments to a function. 
# It is used to pass a non-key worded, variable-length argument list. 

def myFun(arg1, *argv):
    print ("First argument :", arg1)
    for arg in argv:
        print("Next argument through *argv :", arg)
  
myFun('Hello', 'Welcome', 'to', 'GeeksforGeeks')

First argument : Hello
Next argument through *argv : Welcome
Next argument through *argv : to
Next argument through *argv : GeeksforGeeks


In [ ]:
class SoftmaxwithCrossEntropyLoss: #book P140
    def __init__(self):
        self.loss = None
        self.yhat = None
        self.y = None
        
    def forward(self, x, y):
        self.y = y
        self.yhat = softmax(x)
        self.loss = cross_entropy_error(self.yhat, self.y)
        
    def backward(self, dout):
        batch_size = self.y.shape[0]
        if self.y.size == self.yhat.size :  #size1 ??
            dx = (self.yhat - self.y) / batch_size   # 0/1 (pred prob - 1:real)
        else:
            dx = self.yhat.copy()
            dx[np.arange(batch_size), self.y] -= 1
            dx = dx / batch_size
        return dx
    
def softmax(x):  #(n_samples, class_prob)
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

In [12]:
a = np.array([[1,2,3],
              [4,5,6],
              [9,10,11],
              [3,15,18]])
softmax(a)

array([[9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [2.91394551e-07, 4.74258594e-02, 9.52573849e-01]])

In [49]:
def softmax(x):
    print(np.max(x, axis=-1, keepdims=True))
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)

def cross_entropy_error(y, yt):
    if y.ndim == 1:
        yt = yt.reshape(1, yt.size)
        y = y.reshape(1, y.size)
    if yt.size == y.size:
        yt = yt.argmax(axis=1)     
    batch_size = y.shape[0]
    return -np.mean(np.log(y[np.arange(batch_size), yt] + 1e-7))

In [50]:
x= softmax([[1,2,0.9],[1,2,0.1],[1,2,0.0]])
np.max(x, axis=-1, keepdims=True)

[[2.]
 [2.]
 [2.]]


array([[0.58797571],
       [0.65900114],
       [0.66524096]])

In [27]:
# 這個function怪怪的

def one_hot_transformation(y):
    k = len(np.unique(y))
    return np.eye(k)[y]

np.shape(np.eye(5))
np.eye(5)[4]

array([0., 0., 0., 0., 1.])

In [38]:
x = np.array([[1,0,0],
              [0,1,0],
              [0,0,1],
              [0,0,1],
              [1,0,0]])  # (5 data points, 3 cats)
print(x.shape)
print('data count :',x.shape[0])

y = np.array([1,2,3,3,1])  # (5 data points, 3 cats)
print(x.shape)
print('data count :',x.shape[0])

(5, 3)
data count : 5
(5, 3)
data count : 5


In [ ]:
# Ans
class SoftmaxWithCrossEntropyLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, yt):
        self.yt = yt
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.yt)
        return self.loss

    def backward(self, dout):
        batch_size = self.yt.shape[0]
        if self.yt.size == self.y.size: # deal with one-hot-encoding y
            dx = (self.y - self.yt) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.yt] -= 1
            dx = dx / batch_size
        return dx

In [17]:
import numpy as np
x = np.random.randn(4,2)
print(x)
x= np.array([1,2,3,4])
print(x)
y = np.array([1,3,5,7])
print(y)
print(np.dot(x.T,y))

[[-0.92576723  1.05570681]
 [-1.3781495  -0.26838915]
 [ 0.9792404   0.31689814]
 [-0.52831287  1.53056277]]
[1 2 3 4]
[1 3 5 7]
50


In [ ]:
class Linear:
    def __init__(self, w, b):
        self.w = w 
        self.b = b
        self.x = None
        self.x_shape = None
        self.dw = None
        self.db = None
        
    def forward(self, x):  # sum
        self.x_shape = x.shape #?? reshape to (sample_size, list)
        x = x.reshape(x.shape[0], -1)
        self.x = x
        summ = np.dot(self.x, self.w) + self.b
        return summ
    
    def backward(self, dout):  # computational graph reverse method : d_output is known
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = 

In [26]:
# simple experiment
x = np.array([1,11,12,13,2,21,22,32,3,13,23,33])
x = x.reshape((3,2,2))
print(x)
print(x.shape)
y = x.reshape(x.shape[0], -1)
print(y)
print(y.shape)

[[[ 1 11]
  [12 13]]

 [[ 2 21]
  [22 32]]

 [[ 3 13]
  [23 33]]]
(3, 2, 2)
[[ 1 11 12 13]
 [ 2 21 22 32]
 [ 3 13 23 33]]
(3, 4)


In [2]:
from collections import OrderedDict
odd = OrderedDict()

In [4]:
odd['1'] = 'a'
odd['2'] = 'b'
odd['3'] = 'c'
odd['4'] = 'd'

In [10]:
odd.values()

odict_values(['a', 'b', 'c', 'd'])

In [12]:
for od in odd.values() :
    print(od)
    
for od in odd :
    print(od)

a
b
c
d
1
2
3
4
